# Calculo das Proficiencias Individuais

In [1]:
proficiencia = {}
porc_vit = {}

In [2]:
import pandas as pd
import numpy as np
import random as rd
from math import ceil,floor,factorial
from scipy.optimize import minimize
Times = pd.read_excel('TIMES.xlsx')
Times.set_index('Número', inplace = True)
Times.fillna('NA', inplace = True)
Times['Porcentagem de Vitorias'] = pd.Series(np.random.randn(len(list(Times.Sigla))), index=Times.index)
Times_msi = ['IG','G2','SKT','TL','FW','VEG','MEGA','FB','PVB','ITZ','DFM','ISG','BMR']
ACERTOS=0
JOGOS_TOTAIS = 0

In [3]:
def arrumadados(sheet,guia,vero = False):
    arquivo = pd.ExcelFile(sheet)
    liga = pd.read_excel(arquivo,guia-1)
    if not vero:
        liga = liga[3:]
    liga.fillna(0,inplace = True)
    liga.replace(0.7,0.8, inplace = True)
    return liga

In [8]:
liga_time_reg = []
#Declarando as tabelas de jogos de cada regiao
cblol_2019_1 = arrumadados('2019/CBLOL_BRASIL_2019.xlsx',1,True)
a = list(cblol_2019_1.Winner)
a.extend(list(cblol_2019_1.Loser))
times_cblol_2019 = set(a)
liga_time_reg.append((cblol_2019_1,times_cblol_2019,'BR'))

lec_2019_1 = arrumadados('2019/LEC_EUROPA_2019.xlsx',1,True)
a = list(lec_2019_1.Winner)
a.extend(list(lec_2019_1.Loser))
times_lec_2019 = set(a)
liga_time_reg.append((lec_2019_1,times_lec_2019,'EU'))

lcs_2019_1 = arrumadados('2019/LCS_USA_2019.xlsx',1,True)
lcs_2019_1.replace('100',100, inplace = True)
a = list(lcs_2019_1.Winner)
a.extend(list(lcs_2019_1.Loser))
times_lcs_2019 = set(a)
liga_time_reg.append((lcs_2019_1,times_lcs_2019,'NA'))

lck_2019_1 = arrumadados('2019/LCK_COREIA_2019.xlsx',1,True)
a = list(lck_2019_1.Winner)
a.extend(list(lck_2019_1.Loser))
times_lck_2019 = set(a)
liga_time_reg.append((lck_2019_1,times_lck_2019,'KR'))

lpl_2019_1 = arrumadados('2019/LPL_CHINA_2019.xlsx',1,True)
a = list(lpl_2019_1.Winner)
a.extend(list(lpl_2019_1.Loser))
times_lpl_2019 = set(a)
liga_time_reg.append((lpl_2019_1,times_lpl_2019,'CN'))

lms_2019_1 = arrumadados('2019/LMS_TWHKMA_2019.xlsx',1,True)
a = list(lms_2019_1.Winner)
a.extend(list(lms_2019_1.Loser))
times_lms_2019 = set(a)
liga_time_reg.append((lms_2019_1,times_lms_2019,'TWHKMA'))

vcs_2019_1 = arrumadados('2019/VCS_VIETNAM_2019.xlsx',1,True)
a = list(vcs_2019_1.Winner)
a.extend(list(vcs_2019_1.Loser))
times_vcs_2019 = set(a)
liga_time_reg.append((vcs_2019_1,times_vcs_2019,'VN'))

lcl_2019_1 = arrumadados('2019/LCL_CIS_2019.xlsx',1,True)
a = list(lcl_2019_1.Winner)
a.extend(list(lcl_2019_1.Loser))
times_lcl_2019 = set(a)
liga_time_reg.append((lcl_2019_1,times_lcl_2019,'CIS'))

lla_2019_1 = arrumadados('2019/LLA_LA_2019.xlsx',1,True)
a = list(lla_2019_1.Winner)
a.extend(list(lla_2019_1.Loser))
times_lla_2019 = set(a)
liga_time_reg.append((lla_2019_1,times_lla_2019,'LA'))

tcl_2019_1 = arrumadados('2019/TCL_TURQUIA_2019.xlsx',1,True)
a = list(tcl_2019_1.Winner)
a.extend(list(tcl_2019_1.Loser))
times_tcl_2019 = set(a)
liga_time_reg.append((tcl_2019_1,times_tcl_2019,'TRK'))

ljl_2019_1 = arrumadados('2019/LJL_JAPAN_2019.xlsx',1,True)
a = list(ljl_2019_1.Winner)
a.extend(list(ljl_2019_1.Loser))
times_ljl_2019 = set(a)
liga_time_reg.append((ljl_2019_1,times_ljl_2019,'JPN'))

opl_2019_1 = arrumadados('2019/OPL_OCEANIA_2019.xlsx',1,True)
a = list(opl_2019_1.Winner)
a.extend(list(opl_2019_1.Loser))
times_opl_2019 = set(a)
liga_time_reg.append((opl_2019_1,times_opl_2019,'OCEANIA'))

lst_2019_1 = arrumadados('2019/LST_SEA_2019.xlsx',1,True)
a = list(lst_2019_1.Winner)
a.extend(list(lst_2019_1.Loser))
times_lst_2019 = set(a)
liga_time_reg.append((lst_2019_1,times_lst_2019,'SEA'))

In [5]:
def liga_usada(liga,times,sigla):
    global x0,b,bnds
    x0 = list(np.random.uniform(0.5,2.5,len(times)))
    b = (0.001,5.0)
    bnds = tuple([b]*len(times))
    return (liga,times,sigla)

def vero(x):
    y = 0
    proficiencia[sigla] = {}
    for time in times:
        proficiencia[sigla][time] = x[y]
        y+=1
    multiplicatorio = 1
    #Defino a constante de esquecimento k, obtida experimentalmente
    k = 0.075
    for game in range (0,liga.shape[0]):
        w = liga.iloc[game].Week
        pv = proficiencia[sigla][liga.iloc[game].Winner]
        pl = proficiencia[sigla][liga.iloc[game].Loser]
        
        #Diferenciando play-offs de fase regular
        if w>0:
            cte = 1 - (max(list(liga.Week)) - w)*k
        else:
            cte = 1+k
        multiplicatorio *= (pv/(pv+pl))**cte
    
    return -np.log(multiplicatorio)

def probabilidade(A,B,n,provisorio = False):
    A_list = []
    B_list = []
    i,k = 0,floor(n/2)
    if provisorio:
        x = salvar[A]/(salvar[A]+salvar[B])
    else:
        siglaA = list(Times[Times.Sigla==A]['Região'])[0]
        siglaB = list(Times[Times.Sigla==B]['Região'])[0]
        x = proficiencia[siglaA][A]/(proficiencia[siglaA][A]+proficiencia[siglaB][B])
    print("         {} vs {}".format(A,B))
    while (ceil(n/2)+i) <= n:
        if i == 0:
            A_list.append(x**(ceil(n/2))*(1-x)**(i))
            B_list.append(x**(i)*(1-x)**(ceil(n/2)))
        else:
            p = factorial(k)/(factorial(i)*factorial(k-i)) 
            A_list.append(p*x**(ceil(n/2))*(1-x)**(i))
            B_list.append(p*x**(i)*(1-x)**(ceil(n/2)))
        i+=1
        k+=1
    if n==1:
        print('{} tem {:.2f} % de chance de vitória'.format(A,100*sum(A_list)))
        print('Enquanto {} tem {:.2f} %.'.format(B,100*sum(B_list)))
    else:    
        print('{} tem {:.2f} % de chance de vitória.Sendo:'.format(A,100*sum(A_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*A_list[b],ceil(n/2),b))
        print('Enquanto {} tem {:.2f} %. Sendo:'.format(B,100*sum(B_list)))
        for b in range(0,ceil(n/2)):
            print('{:.2f} % a de ser {}-{}'.format(100*B_list[b],ceil(n/2),b))
    
def restricao1(x):
    return x[0]-1        

def restricao2(x):
    return sum(x)-5

def restricao3(x):
    return sum(x)-50

con1 = {'type':'eq','fun':restricao1}
con2 = {'type':'eq','fun':restricao2}
con3 = {'type':'eq','fun':restricao3}
cons = [con2]

In [10]:
#Para calcular a proficiencia de uma liga apenas
liga,times,sigla = liga_usada(lcl_2019_1,times_lcl_2019,'CIS')
sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)
proficiencia[sigla]

{'DA': 0.11660740479998463,
 'EPG': 1.4358170278199949,
 'GMB': 1.099824548812155,
 'M19': 0.18620827128630146,
 'ROX': 0.0773522447205315,
 'TJ': 0.050483892288703106,
 'VEG': 2.032706610334335,
 'VS': 0.001}

In [9]:
#Calculando todas as proficiencias de todos os times de todas as ligas
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    sol = minimize(vero,x0,method = 'SLSQP',constraints = cons,bounds = bnds)

In [10]:
#Pegando os dados de porcentagem de vitoria de todos os times
for trio in liga_time_reg:    
    liga,times,sigla = liga_usada(trio[0],trio[1],trio[2])
    for team in times:
        a = list(liga.Winner)
        a.extend(list(liga.Loser))
        porc_vit[team] = list(liga.Winner).count(team)/a.count(team)

#### Modelo com constante de esquecimento melhor em 66,1 % dos casos
### Previsoes da Liga Coreana
####  Taxa de Acerto na Fase de Grupos: 25/32   78,13%
#### Taxa de Acerto em Play-Offs : 3/4    75%
### Previsoes da Liga Chinesa
####  Taxa de Acerto na Fase de Grupos: 38/48   79,17%
#### Taxa de Acerto em Play-Offs : 4/8 50%
### Previsoes da Liga Norte Americana
####  Taxa de Acerto na Fase de Grupos: 23/31    74,19%
#### Taxa de Acerto em Play-Offs : 5/5 100%
### Previsoes da Liga Europeia
#### Taxa de Acerto na Fase de Grupos: 16/26 61,54%
#### Taxa de Acerto em Play-Offs : 4/6 66,66%
### Previsoes da Liga Brasileira
#### Taxa de Acerto na Fase de Grupos: 29/36    80,55%
#### Taxa de Acerto em Play-Offs : 2/3    66,66%

### -----------------------------------------------------------------------------------------------------------------------------------------------

### Taxa de Acerto Geral na Fase de Grupos: 131/173 75,72%
### Taxa de Acerto Geral em Play-Offs: 24/34 70,59%
### Taxa de Acerto Geral Final: 74,88%

# Calculo da proficiencia relativa entre as regioes

In [11]:
#Trocando os nomes de duas regioes que se juntaram e se tornaram uma so
Times.replace('LAS','LA',inplace = True)
Times.replace('LAN','LA',inplace = True)
#Criando um dicionario com todas as regioes
regions = {}
for reg in set(Times.Região): 
    regions[reg] = list(Times.Sigla[Times.Região == reg])
    
#Criando um com todos os mundiais e importando os respectivos jogos
    mundial = {}
for ano in range (2011,2019):
    mundial[ano] = arrumadados("MUNDIAIS.xlsx",ano-2010,True)

#Criando um dataframe com as regiões e suas proficiências em cada ano
Proficiencias_Relativas = pd.DataFrame(data = None, index = list(set(Times.Região)),columns = [2011,2012,2013,2014,2015,2016,2017,2018,2019])


In [12]:
#Definindo bounds diferentes
bnds_reg = ((0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0),(0.001,50.0))
#Criando uma função que adiciona a proficiência relativa de cada região no dataframe
def adicione(ano):
    m = min(proficiencia.values())
    M = max(proficiencia.values())
    for reg in set(Times.Região):
        if reg in list(proficiencia.keys()):
            Proficiencias_Relativas.loc[[reg],[ano]] = (proficiencia[reg]-m)/(M-m)
            
#Modificando a funcao original pra que se adeque ao objetivo proposto
def vero_reg(x):
    y = 0
    for time in times:
        proficiencia_reg[time] = x[y]
        y+=1
    multiplicatorio = 1
    for ano in range(2011,2019):
        cship = mundial[ano]
        coef = 1 - (2018-ano)/8
        for game in range (0,int(cship.shape[0])):
            pv = proficiencia_reg[cship.iloc[game].Winner]
            pl = proficiencia_reg[cship.iloc[game].Loser]
            multiplicatorio *= (pv/(pv+pl))**(coef)
    
    return -np.log(multiplicatorio)

def probabilidadeR(A,B):
    x = proficiencia_reg[A]/(proficiencia_reg[A]+proficiencia_reg[B])
    print("         {} vs {}".format(A,B))
    print("{} tem {:.2f} % de chance de vitória".format(A,100*x))
    print("Enquanto {} tem {:.2f} %".format(B,100*(1-x)))

In [13]:
for camp in mundial.values():
    for reg in set(Times.Região):
        for linha in range(0,camp.shape[0]):
            if camp.iloc[linha].Winner in regions[reg]:
                camp.iloc[linha].Winner = reg
            if camp.iloc[linha].Loser in regions[reg]:
                camp.iloc[linha].Loser = reg

In [14]:
proficiencia_reg = {}
cons = con3
liga,times,sigla = liga_usada(lck_2019_1,list(set(Times.Região)),'MUND')
sol = minimize(vero_reg,x0,method = 'SLSQP',constraints = cons,bounds = bnds_reg)
proficiencia_reg

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:26: RuntimeWarning: divide by zero encountered in log


{'BR': 0.723127935655326,
 'CIS': 4.478707343325484,
 'CN': 8.514593681807442,
 'EU': 6.554794847028899,
 'JPN': 0.4367357280079335,
 'KR': 14.78690278303663,
 'LA': 1.3650042335260835,
 'NA': 5.2305378554336395,
 'OCEANIA': 0.5006368867294304,
 'SEA': 0.10266678264949869,
 'TRK': 1.735680802885146,
 'TWHKMA': 3.0664623163148454,
 'VN': 2.5041488185008025}

In [15]:
sorted(proficiencia_reg.items(), key=lambda x: x[1])

[('SEA', 0.10266678264949869),
 ('JPN', 0.4367357280079335),
 ('OCEANIA', 0.5006368867294304),
 ('BR', 0.723127935655326),
 ('LA', 1.3650042335260835),
 ('TRK', 1.735680802885146),
 ('VN', 2.5041488185008025),
 ('TWHKMA', 3.0664623163148454),
 ('CIS', 4.478707343325484),
 ('NA', 5.2305378554336395),
 ('EU', 6.554794847028899),
 ('CN', 8.514593681807442),
 ('KR', 14.78690278303663)]

In [16]:
#Fazendo a conversao de medida entre as escalas individuais de cada regiao para a escala global
#Assim, podemos fazer previsoes entre times de outras regioes
for trio in liga_time_reg:    
    sigla = trio[2]    
    top3 = list(proficiencia[sigla].values())
    top3.sort(reverse = True)
    top3 = top3[0:3]
    p_relativa = proficiencia_reg[sigla]
    if sigla=='SEA':
        proficiencia[sigla].update((x,y*p_relativa/top3[0]) for x,y in proficiencia[sigla].items())
    else:
        if sigla in ['EU','NA','CN','KR']:
            harm = 3/(1/top3[0] + 1/top3[1] + 1/top3[2])
        else:
            harm = 2/(1/top3[0] + 1/top3[1])
        proficiencia[sigla].update((x,y*p_relativa/harm) for x,y in proficiencia[sigla].items())

### -----------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
## Criando um dataframe com os times ativos em 2019
listadesiglas = []
for i in range (0,len(proficiencia.keys())):
    for sigla in list(list(proficiencia.values())[i].keys()):
        listadesiglas.append(sigla)
Times_2019 = Times[Times.Sigla.isin(listadesiglas)]
del Times_2019['Força Inicial']
del Times_2019['Força atualizada']
lista2 = []

for time in Times_2019.Sigla:
    lista2.append(proficiencia[list(Times_2019[Times_2019.Sigla==time]['Região'])[0]][time])
    Times_2019.at[Times_2019[Times_2019.Sigla==time].index, 'Porcentagem de Vitorias'] = porc_vit[time]
            
Times_2019['Proficiencia'] = pd.Series(lista2, index=Times_2019.index)
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:int(10000*x))
Times_2019['Porcentagem de Vitorias'] = Times_2019['Porcentagem de Vitorias'].apply(lambda x:x/100)

/usr/local/lib/python3.5/dist-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand

In [18]:
Times_2019[Times_2019.Sigla.isin(Times_msi)].sort_values(by =['Proficiencia'], ascending = False).round(3)

,Sigla,Região,Porcentagem de Vitorias,Proficiencia
Número,,,,
7,SKT,KR,74.00,21.507
21,IG,CN,73.80,11.350
54,G2,EU,79.16,10.528
160,FB,TRK,91.66,6.058
71,TL,NA,76.92,6.019
92,VEG,CIS,76.19,5.028
179,PVB,VN,83.33,4.915
39,FW,TWHKMA,65.00,3.518
98,DFM,JPN,95.83,3.012


In [5]:
salvar = {'SKT':21.507,'IG':11.350,'G2':10.528,'FB':6.058,'TL':6.019,'VEG':5.028,'PVB':4.915,'FW':3.518,'DFM':3.012,'BMR':1.29,'ISG':1.184,'ITZ':0.515,'MEGA':0.103}


# Previsoes no MSI

### Play - In

#### Dia 1

In [26]:
probabilidade('PVB','BMR',1) #OK

         PVB vs BMR
PVB tem 78.74 % de chance de vitória
Enquanto BMR tem 21.26 %.


In [27]:
probabilidade('FB','ISG',1) #OK

         FB vs ISG
FB tem 87.00 % de chance de vitória
Enquanto ISG tem 13.00 %.


In [28]:
probabilidade('PVB','ISG',1) #OK

         PVB vs ISG
PVB tem 80.14 % de chance de vitória
Enquanto ISG tem 19.86 %.


In [29]:
probabilidade('FB','BMR',1) #OK

         FB vs BMR
FB tem 86.00 % de chance de vitória
Enquanto BMR tem 14.00 %.


In [30]:
probabilidade('BMR','ISG',1) #OK

         BMR vs ISG
BMR tem 52.14 % de chance de vitória
Enquanto ISG tem 47.86 %.


In [31]:
probabilidade('FB','PVB',1) #N

         FB vs PVB
FB tem 62.38 % de chance de vitória
Enquanto PVB tem 37.62 %.


In [39]:
probabilidade('BMR','PVB',1) #N

         BMR vs PVB
BMR tem 21.26 % de chance de vitória
Enquanto PVB tem 78.74 %.


In [40]:
probabilidade('ISG','FB',1) #N

         ISG vs FB
ISG tem 13.00 % de chance de vitória
Enquanto FB tem 87.00 %.


#### Dia 2

In [42]:
probabilidade('PVB','ISG',1) #OK

         PVB vs ISG
PVB tem 80.14 % de chance de vitória
Enquanto ISG tem 19.86 %.


In [44]:
probabilidade('FB','BMR',1) #OK

         FB vs BMR
FB tem 86.00 % de chance de vitória
Enquanto BMR tem 14.00 %.


In [45]:
probabilidade('PVB','FB',1) #OK

         PVB vs FB
PVB tem 37.62 % de chance de vitória
Enquanto FB tem 62.38 %.


In [43]:
probabilidade('ISG','BMR',1) #OK (isurus ganhou)

         ISG vs BMR
ISG tem 47.86 % de chance de vitória
Enquanto BMR tem 52.14 %.


In [33]:
probabilidade('MEGA','DFM',1) #OK

         MEGA vs DFM
MEGA tem 3.31 % de chance de vitória
Enquanto DFM tem 96.69 %.


In [34]:
probabilidade('VEG','ITZ',1) #OK

         VEG vs ITZ
VEG tem 90.77 % de chance de vitória
Enquanto ITZ tem 9.23 %.


In [36]:
probabilidade('VEG','DFM',1) #OK

         VEG vs DFM
VEG tem 62.63 % de chance de vitória
Enquanto DFM tem 37.37 %.


In [41]:
probabilidade('MEGA','ITZ',1) #N

         MEGA vs ITZ
MEGA tem 16.74 % de chance de vitória
Enquanto ITZ tem 83.26 %.


In [53]:
probabilidade('PVB','FB',1) #N

         PVB vs FB
PVB tem 37.62 % de chance de vitória
Enquanto FB tem 62.38 %.


#### Dia 3

In [37]:
probabilidade('DFM','ITZ',1) #OK

         DFM vs ITZ
DFM tem 85.45 % de chance de vitória
Enquanto ITZ tem 14.55 %.


In [38]:
probabilidade('VEG','MEGA',1) #OK

         VEG vs MEGA
VEG tem 98.00 % de chance de vitória
Enquanto MEGA tem 2.00 %.


In [46]:
probabilidade('DFM','MEGA',1) #OK

         DFM vs MEGA
DFM tem 96.69 % de chance de vitória
Enquanto MEGA tem 3.31 %.


In [47]:
probabilidade('ITZ','VEG',1) #OK

         ITZ vs VEG
ITZ tem 9.23 % de chance de vitória
Enquanto VEG tem 90.77 %.


In [48]:
probabilidade('ITZ','MEGA',1) #OK

         ITZ vs MEGA
ITZ tem 83.26 % de chance de vitória
Enquanto MEGA tem 16.74 %.


In [50]:
probabilidade('DFM','VEG',1) #OK

         DFM vs VEG
DFM tem 37.37 % de chance de vitória
Enquanto VEG tem 62.63 %.


In [51]:
probabilidade('ITZ','DFM',1) #OK

         ITZ vs DFM
ITZ tem 14.55 % de chance de vitória
Enquanto DFM tem 85.45 %.


In [54]:
probabilidade('MEGA','VEG',1) #N

         MEGA vs VEG
MEGA tem 2.00 % de chance de vitória
Enquanto VEG tem 98.00 %.


#### Dia 4 

In [56]:
probabilidade('TL','PVB',5) #OK

         TL vs PVB
TL tem 60.80 % de chance de vitória.Sendo:
17.39 % a de ser 3-0
23.05 % a de ser 3-1
20.37 % a de ser 3-2
Enquanto PVB tem 39.20 %. Sendo:
8.63 % a de ser 3-0
14.45 % a de ser 3-1
16.13 % a de ser 3-2


In [58]:
probabilidade('FW','VEG',5) #N

         FW vs VEG
FW tem 33.79 % de chance de vitória.Sendo:
6.98 % a de ser 3-0
12.32 % a de ser 3-1
14.49 % a de ser 3-2
Enquanto VEG tem 66.21 %. Sendo:
20.36 % a de ser 3-0
25.15 % a de ser 3-1
20.71 % a de ser 3-2


In [19]:
probabilidade('VEG','PVB',5) #OK (PVB ganhou)

         VEG vs PVB
VEG tem 51.07 % de chance de vitória.Sendo:
12.93 % a de ser 3-0
19.18 % a de ser 3-1
18.96 % a de ser 3-2
Enquanto PVB tem 48.93 %. Sendo:
12.08 % a de ser 3-0
18.32 % a de ser 3-1
18.53 % a de ser 3-2


### Group Stage

#### Dia 1

In [20]:
probabilidade('SKT','G2',1) #N

         SKT vs G2
SKT tem 67.14 % de chance de vitória
Enquanto G2 tem 32.86 %.


In [21]:
probabilidade('FW','TL',1) #OK

         FW vs TL
FW tem 36.89 % de chance de vitória
Enquanto TL tem 63.11 %.


In [22]:
probabilidade('PVB','IG',1) #OK

         PVB vs IG
PVB tem 30.22 % de chance de vitória
Enquanto IG tem 69.78 %.


In [23]:
probabilidade('SKT','FW',1) #OK

         SKT vs FW
SKT tem 85.94 % de chance de vitória
Enquanto FW tem 14.06 %.


In [24]:
probabilidade('IG','G2',1) #OK

         IG vs G2
IG tem 51.88 % de chance de vitória
Enquanto G2 tem 48.12 %.


In [9]:
probabilidade('TL','PVB',1) #OK

         TL vs PVB
TL tem 55.05 % de chance de vitória
Enquanto PVB tem 44.95 %.


#### Dia 2

In [10]:
probabilidade('G2','TL',1,True) #OK

         G2 vs TL
G2 tem 63.62 % de chance de vitória
Enquanto TL tem 36.38 %.


In [11]:
probabilidade('PVB','FW',1,True) #N

         PVB vs FW
PVB tem 58.28 % de chance de vitória
Enquanto FW tem 41.72 %.


In [12]:
probabilidade('IG','TL',1,True) #OK

         IG vs TL
IG tem 65.35 % de chance de vitória
Enquanto TL tem 34.65 %.


In [13]:
probabilidade('SKT','PVB',1,True) #OK

         SKT vs PVB
SKT tem 81.40 % de chance de vitória
Enquanto PVB tem 18.60 %.


In [14]:
probabilidade('G2','FW',1,True) #OK

         G2 vs FW
G2 tem 74.95 % de chance de vitória
Enquanto FW tem 25.05 %.


In [16]:
probabilidade('IG','SKT',1,True) #N

         IG vs SKT
IG tem 34.54 % de chance de vitória
Enquanto SKT tem 65.46 %.


#### Dia 4

In [17]:
probabilidade('TL','SKT',1,True) #OK

         TL vs SKT
TL tem 21.87 % de chance de vitória
Enquanto SKT tem 78.13 %.


In [18]:
probabilidade('IG','FW',1,True) #OK

         IG vs FW
IG tem 76.34 % de chance de vitória
Enquanto FW tem 23.66 %.


In [19]:
probabilidade('PVB','G2',1,True) #N

         PVB vs G2
PVB tem 31.83 % de chance de vitória
Enquanto G2 tem 68.17 %.


In [20]:
probabilidade('TL','FW',1,True) #N

         TL vs FW
TL tem 63.11 % de chance de vitória
Enquanto FW tem 36.89 %.


In [21]:
probabilidade('SKT','G2',1,True) #N

         SKT vs G2
SKT tem 67.14 % de chance de vitória
Enquanto G2 tem 32.86 %.


In [22]:
probabilidade('IG','PVB',1,True) #OK

         IG vs PVB
IG tem 69.78 % de chance de vitória
Enquanto PVB tem 30.22 %.
